In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/Users/suprita/Desktop/supriML/kaggle_NLP/nlp-getting-started/train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [16]:
x, y = data.iloc[:, :-1], data.iloc[:, -1]
print(x.head())
print(y.head())
# LATER TEST: will dropping the index col affect anything
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

   id keyword location                                               text
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...
0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64


In [50]:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from transformers import AutoTokenizer
import re



class processText(BaseEstimator, TransformerMixin):
    def __init__(self): 
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


    def fit(self, x, y=None): 
        return self

    def clean_and_tokenize(self, text):
        text = text.lower()
        text = ''.join([c for c in text if c.isdigit()])
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return ' '.join(self.tokenizer.tokenize(' '.join(tokens)))


    def transform(self, x, y=None): 
        return x.apply(self.clean_and_tokenize)

# Create the pipeline
pipeline = Pipeline([
    ('text_processor', processText()),  # Use the updated processText class
    ('vectorizer', TfidfVectorizer(max_features=5000)),
    ('model', MultinomialNB())
])

# Fit the pipeline to the training data
pipeline.fit(xtrain['text'], ytrain)

/Users/suprita/Desktop/supriML/kaggle_NLP/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Pipeline(steps=[('text_processor', processText()),
                ('vectorizer', TfidfVectorizer(max_features=5000)),
                ('model', MultinomialNB())])

In [51]:
# test model
from sklearn.metrics import accuracy_score
ypred = pipeline.predict(xtest['text'])
accuracy_score(ytest, ypred)



0.5738673670387393

In [60]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
import tensorflow as tf

# Load the tokenizer and model specifically designed for TensorFlow
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = TFAutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Function to tokenize the input data
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', return_tensors='tf')

# Assuming 'data' is a pandas DataFrame with 'text' and 'label' columns
dataset = Dataset.from_pandas(data)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split the dataset into training and testing sets
train_dataset, test_dataset = tokenized_dataset.train_test_split(test_size=0.2).values()

# Convert datasets to TensorFlow Dataset objects
train_tf_dataset = train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle=True,
    batch_size=8
)

test_tf_dataset = test_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle=False,
    batch_size=8
)

# Define the training arguments
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# Compile the model
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

# Train the model
model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=3)

# Evaluate the model
results = model.evaluate(test_tf_dataset)
print(f"Test Accuracy: {results[1] * 100:.2f}%")

ValueError: object __array__ method not producing an array